In [1]:
import pandas as pd
import numpy as np
import os
from PIL import Image

In [2]:
# Step 1: Load the Data from CSV
csv_file = "clean_MovieGenre.csv"  # Path to your CSV file
df = pd.read_csv(csv_file)

# Specify the binarized columns
binarized_columns = ['Action', 'Adventure', 'Animation', 'Biography', 'Comedy', 'Crime', 'Documentary', 
                     'Drama', 'Family', 'Fantasy', 'Film-Noir', 'History', 'Horror', 'Music', 'Musical', 
                     'Mystery', 'Romance', 'Sci-Fi', 'Short', 'Sport', 'Thriller', 'War', 'Western']


In [3]:
# Step 2: Image Loading
image_folder = "downloaded_images"  # Path to the folder containing the images
images = []
labels = []

In [6]:
for index, row in df.iterrows():
    # Get the filename and labels from the CSV
    filename = str(row[0]) + ".jpg"  # Assuming the filenames match the imdbIDs
    label = [int(row[column]) for column in binarized_columns]  # Extract binarized labels for each genre

    image_path = os.path.join(image_folder, filename)
    try:
        image = Image.open(image_path)  # Open image using PIL
        image = image.convert("RGB")  # Convert image to RGB mode if necessary
        image_array = np.array(image)  # Convert PIL Image to numpy array
        images.append(image_array)
        labels.append(label)
    except Exception as e:
        print(f"Error loading image {filename}: {e}")

/var/folders/_2/9tdd0_v908qf2_1bw_19j5wr0000gn/T/ipykernel_95406/2721500586.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  filename = str(row[0]) + ".jpg"  # Assuming the filenames match the imdbIDs


In [7]:
# Convert lists to numpy arrays for further processing
images = np.array(images)
labels = np.array(labels)

# Print the shapes of the loaded data
print("Images shape:", images.shape)
print("Labels shape:", labels.shape)

Images shape: (33275, 268, 182, 3)
Labels shape: (33275, 23)
